# Elliptic Mayo Writeup

## Our Domain Parameters and Public Information

Given we are working with Elliptic Curves, any valid point $(x, y)$ will satisfy the following equality

$$
y^2 = x^3 + ax + b,
$$

where arithmetic is done modulo $p$. Fortunately, this information is given in the instructions as



In [2]:
p = 0xdb8f1e4884c47bfb
a = 0xdb8f1e4884c47bf8
b = 0xba0adf33491811a8

Even further, Computer Algebra Systems like SageMath can do a lot of heavy lifting for us mathematically as concepts like Elliptic Curves are already built in. Using our parameters above, we define our elliptic curve $E$ like so:


In [3]:
E = EllipticCurve(GF(p), (a, b))

Note the first argument 'GF\(p\)' simply tells SageMath that we are performing arithmetic modulo $p$. More formally, we are defining our elliptic curve over the finite field of order $p$; this finite field is often denoted as $\mathrm{GF}(p)$.

Furthermore, we are given the following point of our curve $E$:


In [4]:
G = E(0x18c87d6cc12ee703, 0x869a10ce9f08ed34)

We can also test that this point is valid:


In [5]:
x = G.xy()[0]
y = G.xy()[1]
y^2 % p == (x^3 + a*x + b) % p

True

While we do not know Alexandria's private key $d_A$ and Bobicus' private key $d_B$, we do know their public keys $Q_A = d_A \cdot G$ and $Q_B=d_B \cdot G:$ 


In [6]:
Q_A = E(0x28aa430008a24715, 0x537c24c86f3f17a4)
Q_B = E(0x1ce10c7c5989866e, 0x176acbd73cf15bc8)

To find the shared secret $S=d_A \cdot d_B \cdot G$, we need to find either $d_A$ or $d_B$.  Given we have more intel on Alexandria's side, it would be easiest to find $d_A$.



## Invalid Curve Attack

In our true curve $E$, we see that the group of all valid points \(including the neutral point, or point at infinity\) has prime order:


In [7]:
n = E.order()
print('|E| =', n)
n.is_prime()

|E| = 15820897315794384701


True

Let $|E|=n$. Seeing that $G$ is not the identity element, we know $|G| = n$ by Lagrange's theorem. This makes the discrete logarithm problem particularly difficult as $G$ generates the entire group. Naively, we could test every integer between $1$ and $n$ to find $d_A$:


In [8]:
1 * G == Q_A # False
2 * G == Q_A # False
3 * G == Q_A # False
print('This will take awhile.')

This will take awhile.


However, even if we loop or use the built\-in 'discrete\_log' method, we are unlikely to find $d_A$ in a reasonable amount of time. This is where our invalid curves come into play. Fortunately, some of the work has been done for us.

Taking a look at Intel.csv, we have:



In [9]:
import pandas as pd

df = pd.read_csv('Intel.csv')
df

,p,a,b,G,d_A*G,d_B*G
0,0xdb8f1e4884c47bfb,0xdb8f1e4884c47bf8,0xba0adf33491811a8,"(0x18c87d6cc12ee703, 0x869a10ce9f08ed34)","(0x28aa430008a24715, 0x537c24c86f3f17a4)","(0x1ce10c7c5989866e, 0x176acbd73cf15bc8)"
1,0xdb8f1e4884c47bfb,0xdb8f1e4884c47bf8,0xed0e0818000417c8,"(0x6a71de4cf2c7bc02, 0x552e50cbc80691a7)","(0x7fe76431e04a0aaa, 0x8b99f0d205e1aa6)",UNKNOWN
2,0xdb8f1e4884c47bfb,0xdb8f1e4884c47bf8,0xd33e7fc2ba39f952,"(0x20b95fb49c0abba3, 0x86a9d9e1156788e4)","(0xc37fca91993c3e76, 0xcf780d75c662fa11)",UNKNOWN


### Invalid Curve 1

Note that the first row describes our true curve $E$. For our invalid curve $E_1$ \(middle row\), we see that $p$ and $a$ remain the same, but $b$ is altered. Given point addition does not rely on the value for $b$, we may use this invalid curve to narrow our options for $d_A$. We then define $E_1$ like so:



In [10]:
b1 = 0xed0e0818000417c8
E1 = EllipticCurve(GF(p), (a, b1))

We also have a different point $G_1$ and $Q_{1_A}=d_A \cdot G_1$ , where


In [11]:
G1 = E1(0x6a71de4cf2c7bc02, 0x552e50cbc80691a7)
Q1_A = E1(0x7fe76431e04a0aaa, 0x8b99f0d205e1aa6)

However, notice this time that $G_1$ does not generate $E_1$:


In [12]:
print('|E1| =', E1.order())
print('|G1| =', G1.order())

|E1| = 15820897316377355766
|G1| = 66820844767


Moreso, $G_1$ even has prime order:


In [13]:
G1.order().is_prime()

True

Given $|G_1|$ is relatively small, we may calculate the discrete log rather quickly:


In [14]:
G1.discrete_log(Q1_A)

13087305027

This ultimately tells us $13087305027 \cdot G_1 = Q_{1_A}$. However, notice that 

$$
|G_1| = 66820844767.
$$

Treating $\mathcal{O}$ as the identity element of $E_1$, we also see that for any integer \$k\$,

\begin{align}
(13087305027 + k \cdot 66820844767) \cdot G_1 &= 13087305027 \cdot G_1 + k \cdot 66820844767 \cdot G_1 \\
&= Q_{1_A} + k \cdot \mathcal{O} \\
&= Q_{1_A} + \mathcal{O} \\
&= Q_{1_A}. \\
\end{align}

That is to say, we can not yet guarantee a value for $d_A$. We can only assert that

$$
d_A = 13087305027 + k \cdot 66820844767
$$

for some integer $k$. In other words,

$$
d_A \equiv 13087305027 \textrm{ }(\textrm{mod } 66820844767).
$$

We will want to save this information for later.



In [15]:
modulus1 = G1.order()
remainder1 = G1.discrete_log(Q1_A)
remainder1, modulus1

(13087305027, 66820844767)

### Invalid Curve 2

For our next invalid curve \(parameters in last row of Intel.csv\), we proceed analogously to before:



In [16]:
b2 = 0xd33e7fc2ba39f952
E2 = EllipticCurve(GF(p), (a, b2))
G2 = E2(0x20b95fb49c0abba3, 0x86a9d9e1156788e4)
Q2_A = E2(0xc37fca91993c3e76, 0xcf780d75c662fa11)
modulus2 = G2.order()
remainder2 = G2.discrete_log(Q2_A)
remainder2, modulus2

(4614858334, 44177846483)

Note that $|G_2|$ is prime. Ultimately, we gather that

$$
d_A \equiv 4614858334 \textrm{ }(\textrm{mod } 44177846483).
$$

Seeing that $|G_1|$ and $|G_2|$ are prime, we know they are relatively prime. With this information, we can then use the Chinese remainder theorem:



In [17]:
crt(remainder1, remainder2, modulus1, modulus2)

10967933313814980060

That is,


$$
d_A \equiv 10967933313814980060 \textrm{ }(\textrm{mod } 2952001021980899904461)
$$



Seeing that the product of our two moduli—$|G_1|$ and $|G_2|$—is greater than $n$, we have

$$
d_A \equiv 10967933313814980060 \textrm{ }(\textrm{mod } n).
$$



Given $d_A$ takes a value between $1$ and $n-1$, we have


$$
d_A = 10967933313814980060.
$$



With this information, we may now find the shared secret $S$:


In [18]:
d_A = crt(remainder1, remainder2, modulus1, modulus2)
S = d_A * Q_B
S.xy()

(8716978289614203805, 7923394114363233555)

## Decrypting the Cipher

Now that we know the shared secret $S$, we can use its coordinates to find the key and initialization vector to decrypt through AES\-128:


In [19]:
from Crypto.Cipher import AES

x = int(S.xy()[0])
y = int(S.xy()[1])
K = int((x << 64) ^^ y).to_bytes(16, 'big') # ^^ means XOR as ^ was reassigned to exponentiation in SageMath
V = int((y << 64) ^^ x).to_bytes(16, 'big')
cipher = AES.new(K, 2, V) # 2 means CBC mode
ciphertext = int(0x01c62e810475ee812688c2ef10bdd5cfe3bceb68d6ffbb2ee1d1d5d1b2653274).to_bytes(32, 'big')
cipher.decrypt(ciphertext)

b'cygame{7H13r_CuRv3_G4m3_W3ak}\x03\x03\x03'

Considering our padding method, we finally get the flag:

    cygame{7H13r_CuRv3_G4m3_W3ak}.

